<a href="https://colab.research.google.com/github/FawzElHouda/Recommendation_System_Ecommerce/blob/main/Recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import re
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
amazon = pd.read_csv('/content/amazon.csv')
comm = pd.read_csv('/content/ratings_Electronics (1).csv')



In [4]:
amazon.head(2)

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...


In [5]:
amazon.shape

(1465, 16)

In [6]:
comm.head()

,AKM1MP6P0OYPR,0132793040,5.0,1365811200
0,A2CX7LUOHB2NDG,0321732944,5.0,1.341101e+09
1,A2NWSAGRHCP8N5,0439886341,1.0,1.367194e+09
2,A2WNBOD3WNDNKT,0439886341,3.0,1.374451e+09
3,A1GI0U4ZRJA8WN,0439886341,1.0,1.334707e+09
4,A1QGNMC6O1VW39,0511189877,5.0,1.397434e+09


In [7]:
comm.shape

(4942195, 4)

In [8]:
print(amazon.columns)
print(comm.columns)


Index(['product_id', 'product_name', 'category', 'discounted_price',
       'actual_price', 'discount_percentage', 'rating', 'rating_count',
       'about_product', 'user_id', 'user_name', 'review_id', 'review_title',
       'review_content', 'img_link', 'product_link'],
      dtype='object')
Index(['AKM1MP6P0OYPR', '0132793040', '5.0', '1365811200'], dtype='object')


In [9]:
comm = comm.rename(columns={'AKM1MP6P0OYPR': 'user_id'})

comm = comm.rename(columns={'0132793040': 'product_id'})

comm = comm.rename(columns={'5.0': 'rating'})

comm = comm.rename(columns={'1365811200': 'timestamp'})




In [10]:
print(amazon.columns)
print(comm.columns)

Index(['product_id', 'product_name', 'category', 'discounted_price',
       'actual_price', 'discount_percentage', 'rating', 'rating_count',
       'about_product', 'user_id', 'user_name', 'review_id', 'review_title',
       'review_content', 'img_link', 'product_link'],
      dtype='object')
Index(['user_id', 'product_id', 'rating', 'timestamp'], dtype='object')


In [11]:
data = amazon[['product_id', 'product_name', 'category', 'about_product', 'review_title', 'review_content']].copy()

# 2. Gestion des valeurs manquantes (NaN)


In [12]:
print("Nombre de NaN avant :")
print(data.isnull().sum())

Nombre de NaN avant :
product_id        0
product_name      0
category          0
about_product     0
review_title      0
review_content    0
dtype: int64


In [13]:
# Remplacer les NaN dans les champs de texte par une chaîne vide pour éviter de supprimer trop de données
data.fillna('', inplace=True)
print("\nNombre de NaN après :")
print(data.isnull().sum())
data.dropna(inplace=True) # S'il reste des lignes avec NaN ailleurs


Nombre de NaN après :
product_id        0
product_name      0
category          0
about_product     0
review_title      0
review_content    0
dtype: int64


In [14]:
def remove_space(text_list):
    # Assurez-vous que l'entrée est une liste de chaînes
    if isinstance(text_list, str):
        text_list = text_list.split()

    L1 = []
    for i in text_list:
        # Supprime les caractères non alphanumériques et les espaces
        clean_word = re.sub(r'[^a-zA-Z0-9\s]', '', i).replace(" ", "")
        if clean_word:
            L1.append(clean_word)
    return L1

In [15]:
data['category'] = data['category'].apply(lambda x: [x.replace(" ", "")])
data['product_name_words'] = data['product_name'].apply(lambda x: remove_space(x.lower().split()))

In [16]:
data['about_product'] = data['about_product'].apply(lambda x: remove_space(x.lower().split()))
data['review_title'] = data['review_title'].apply(lambda x: remove_space(x.lower().split()))
data['review_content'] = data['review_content'].apply(lambda x: remove_space(x.lower().split()))

In [17]:
print("\nDataFrame après la préparation des colonnes de tags :")
print(data[['product_name_words', 'category', 'about_product', 'review_content']].head(2).to_markdown(index=False))


DataFrame après la préparation des colonnes de tags :
| product_name_words                                                                                                                                                                                                                                                          | category                                                                              | about_product                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [18]:
data['tags'] = (
    data['product_name_words'] +
    data['category'] +
    data['about_product'] +
    data['review_title'] +
    data['review_content']
)

In [19]:
new_df = data[['product_id', 'product_name', 'tags']].copy()

In [20]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

In [21]:
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

In [22]:
ps = PorterStemmer()

def stems(text):
    T = []
    for i in text.split():
        T.append(ps.stem(i))
    return " ".join(T)

In [23]:
new_df['tags'] = new_df['tags'].apply(stems)

In [24]:
print("\nExemple de Tags après Stemming :")
print(new_df.iloc[0]['tags'][:300] + '...')


Exemple de Tags après Stemming :
wayona nylon braid usb to lightn fast charg and data sync cabl compat for iphon 13 1211 x 8 7 6 5 ipad air pro mini 3 ft pack of 1 grey computers&accessories|accessories&peripherals|cables&accessories|cables|usbc high compat compat with iphon 12 11 xxsmaxxr iphon 88 plusiphon 77 plusiphon 6s6 plusip...


In [25]:
cv = CountVectorizer(max_features=5000, stop_words='english')
vector = cv.fit_transform(new_df['tags']).toarray()

In [26]:
print("\nForme du vecteur (nombre de produits, nombre de fonctionnalités/mots-clés) :")
print(vector.shape) # Ex: (4809, 5000)


Forme du vecteur (nombre de produits, nombre de fonctionnalités/mots-clés) :
(1465, 5000)


In [27]:
# 10. Calcul de la similarité cosinus
# Cette mesure de distance est au cœur de l'approche de recommandation basée sur le contenu.
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [28]:
print("\nForme de la matrice de similarité (produits x produits) :")
print(similarity.shape)
# Ex: (4809, 4809)


Forme de la matrice de similarité (produits x produits) :
(1465, 1465)


In [29]:
def recommend(product_name):
    # Trouver l'index du produit
    try:
        index = new_df[new_df['product_name'] == product_name].index[0]
    except IndexError:
        print(f"Produit non trouvé : {product_name}")
        return

    # Calculer les distances et trier
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])

    # Afficher les 5 produits les plus similaires (sauf le produit lui-même)
    print(f"\nRecommandations pour '{product_name}' :")
    for i in distances[1:6]:
        print(f"* {new_df.iloc[i[0]].product_name}")

In [30]:
print(new_df['product_name'].sample(5).tolist())

['Redmi Note 11 (Space Black, 4GB RAM, 64GB Storage)|90Hz FHD+ AMOLED Display | Qualcomm® Snapdragon™ 680-6nm | 33W Charger Included', 'Boult Audio Truebuds with 30H Playtime, IPX7 Waterproof, Lightning Boult™ Type C Fast Charging (10 Min=100Mins), BoomX™ Tech Rich Bass, Pro+ Calling HD Mic, Touch Controls in Ear Earbuds TWS (Grey)', '3M Scotch Double Sided Heavy Duty Tape(1m holds 4.5Kgs) for indoor hanging applications (Photo frames, Mirrors, Key Holders, Car Interiors, Extension Boards, Wall decoration, etc)(L: 3m, W: 24mm)', 'pTron Solero TB301 3A Type-C Data and Fast Charging Cable, Made in India, 480Mbps Data Sync, Strong and Durable 1.5-Meter Nylon Braided USB Cable for Type-C Devices for Charging Adapter (Black)', 'Casio FX-991ES Plus-2nd Edition Scientific Calculator, Black']


In [31]:
recommend('OnePlus')

Produit non trouvé : OnePlus Nord CE 2 Lite 5G


In [35]:
recommend('Redmi Note 11 (Space Black, 4GB RAM, 64GB Storage)|90Hz FHD+ AMOLED Display | Qualcomm® Snapdragon™ 680-6nm | 33W Charger Included')


Recommandations pour 'Redmi Note 11 (Space Black, 4GB RAM, 64GB Storage)|90Hz FHD+ AMOLED Display | Qualcomm® Snapdragon™ 680-6nm | 33W Charger Included' :
* Redmi Note 11 (Space Black, 6GB RAM, 64GB Storage) | 90Hz FHD+ AMOLED Display | Qualcomm® Snapdragon™ 680-6nm | 33W Charger Included
* Redmi Note 11 (Horizon Blue, 6GB RAM, 64GB Storage)|90Hz FHD+ AMOLED Display | Qualcomm® Snapdragon™ 680-6nm | 33W Charger Included
* Redmi Note 11 (Horizon Blue, 6GB RAM, 64GB Storage)|90Hz FHD+ AMOLED Display | Qualcomm® Snapdragon™ 680-6nm | 33W Charger Included
* Redmi Note 11 Pro + 5G (Stealth Black, 6GB RAM, 128GB Storage) | 67W Turbo Charge | 120Hz Super AMOLED Display | Additional Exchange Offers | Charger Included
* Redmi Note 11 Pro + 5G (Phantom White, 8GB RAM, 128GB Storage) | 67W Turbo Charge | 120Hz Super AMOLED Display | Additional Exchange Offers | Charger Included


In [42]:
import pickle


In [43]:
pickle.dump(new_df, open('artifacts/product_list.pkl', 'wb'))
pickle.dump(similarity, open('artifacts/similarity_matrix.pkl', 'wb'))